# Define paramters

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
import v3io_frames as v3f
from os import environ
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

version = '1.0'
#environ['MODEL_FILE'] = 'saved_model.pb'.format(version)
environ['MODEL_FILE'] = 'saved_model.pb

image_size = 160 # All images will be resized to 160x160
IMG_SHAPE = (image_size, image_size, 3)
batch_size = 32


zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip", 
                                   fname="cats_and_dogs_filtered.zip", extract=True)
base_dir, _ = os.path.splitext(zip_file)
validation_dir = os.path.join(base_dir, 'validation')
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")

# Load Model from saved path

In [ ]:

new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.trainable = True
new_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
new_model.summary() #just to verify the model is ok

In [ ]:
#and check the image aginst the model 

In [ ]:
#need to load the data from feed (HTTP) to datagen variable and rescale.
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255) 

# Flow validation images in batches of 20 using test_datagen generator
prediction = datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

# just in case this is more than one image... retuen the item Cat or Dog 
for item in prediction.classes:
    print('Image is a ' + {False: 'Cat', True :'Dog'}[item])